In [1]:
#Generate dataset
import cv2
import csv
import glob

header  =["label"]
for i in range(0,784):
    header.append("pixel"+str(i))
with open('dataset3.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(header)#to append the values in a row
    
for label in range(10):
    dirList = glob.glob("captured_images/"+str(label)+"/*.jpg")#extraction of images
    
    for img_path in dirList:
        im= cv2.imread(img_path)#to read the image
        im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)#to change in grayscale
        im_gray = cv2.GaussianBlur(im_gray,(15,15), 0)#to blur the image
        roi= cv2.resize(im_gray,(28,28), interpolation=cv2.INTER_AREA)#resize
        
        data=[]
        data.append(label)
        rows, cols = roi.shape
        
        ## Add pixel one by one into data array
        for i in range(rows):
            for j in range(cols):
                k =roi[i,j]
                if k>100:
                    k=1
                else:
                    k=0
                data.append(k)
        with open('dataset3.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(data)

In [1]:
#load the dataset
import pandas as pd
from sklearn.utils import shuffle
data  =pd.read_csv('dataset3.csv',low_memory=False)
data=shuffle(data)
data

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
4        0       0       0       0       0       0       0       0       0   
203      2       0       0       0       0       0       0       0       0   
663      6       0       0       0       0       0       0       0       0   
510      5       0       0       0       0       0       0       0       0   
722      7       0       0       0       0       0       0       0       0   
..     ...     ...     ...     ...     ...     ...     ...     ...     ...   
714      7       0       0       0       0       0       0       0       0   
522      5       0       0       0       0       0       0       0       0   
629      6       0       0       0       0       0       0       0       0   
335      3       0       0       0       0       0       0       0       0   
824      8       0       0       0       0       0       0       0       0   

     pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
4         0  ...         0         0         0         0         0         0   
203       0  ...         0         0         0         0         0         0   
663       0  ...         0         0         0         0         0         0   
510       0  ...         0         0         0         0         0         0   
722       0  ...         0         0         0         0         0         0   
..      ...  ...       ...       ...       ...       ...       ...       ...   
714       0  ...         0         0         0         0         0         0   
522       0  ...         0         0         0         0         0         0   
629       0  ...         0         0         0         0         0         0   
335       0  ...         0         0         0         0         0         0   
824       0  ...         0         0         0         0         0         0   

     pixel780  pixel781  pixel782  pixel783  
4           0         0         0         0  
203         0         0         0         0  
663         0         0         0         0  
510         0         0         0         0  
722         0         0         0         0  
..        ...       ...       ...       ...  
714         0         0         0         0  
522         0         0         0         0  
629         0         0         0         0  
335         0         0         0         0  
824         0         0         0         0  

[1000 rows x 785 columns]

In [2]:
#separation of dependent and independent variable
X = data.drop(["label"],axis=1)
Y= data["label"]
X.shape

(1000, 784)

In [3]:
%matplotlib inline

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import cv2

In [8]:
idx = 684
img = X.loc[idx].values.reshape(28,28)
print(Y[idx])#reshape to plot

6


In [9]:
plt.imshow(img)


In [10]:
#Train-Test split
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(X,Y, test_size = 0.2)

In [11]:
#Fit the model using svc and also to save the model using joblib
import joblib
from sklearn.svm import SVC
classifier=SVC(kernel="linear",random_state=6)
classifier.fit(train_x,train_y)
joblib.dump(classifier, "model/hand_digit_recognizer")

['model/hand_digit_recognizer']

In [13]:
#calculate accuracy
from sklearn import metrics
prediction=classifier.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))

Accuracy=  0.545


In [ ]:
#prediction of image drawn in paint

import joblib
import cv2
import numpy as np #pip install numpy
import time
import pyscreenshot as ImageGrab

model=joblib.load("model/hand_digit_recognizer")
images_folder="img/"

while True:
    img=ImageGrab.grab(bbox=(60,170,400,500))
    
    img.save(images_folder+"img.jpg")
    im = cv2.imread(images_folder+"img.jpg")
    im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    im_gray  =cv2.GaussianBlur(im_gray, (15,15), 0)
    
    #Threshold the image
    ret, im_th = cv2.threshold(im_gray,100, 255, cv2.THRESH_BINARY)
    roi = cv2.resize(im_th, (28,28), interpolation  =cv2.INTER_AREA)
    
    rows,cols=roi.shape
    
    X = []
    
    ## Add pixel one by one into data array
    for i in range(rows):
        for j in range(cols):
            k = roi[i,j]
            if k>100:
                k=1
            else:
                k=0
            X.append(k)
            
    predictions  =model.predict([X])
    print("Prediction:",predictions[0])
    cv2.putText(im, "Prediction is: "+str(predictions[0]), (20,20), 0, 0.8,(0,255,0),2,cv2.LINE_AA)
    
    cv2.startWindowThread()
    cv2.namedWindow("Result")
    cv2.imshow("Result",im)
    cv2.waitKey(10000)
    if cv2.waitKey(1)==13: #27 is the ascii value of esc, 13 is the ascii value of enter
        break
cv2.destroyAllWindows()